# Sentiment Classification Multilabel with AutoML for NLP

In [1]:
import pandas as pd
import numpy as np
import json
import requests
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input

from azure.ai.ml import automl
import re

pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)

# 1. Retrieve and preprocess data

## Option A: for Sodexo Input

In [7]:
'''
sentiment_df = pd.read_excel('../../input_data/validation_set_topic_classification_advanced_task.xls')

topic_cols = ['main_topic_1', 'main_topic_2', 'main_topic_3','secondary_topic_1', 'secondary_topic_2', 'secondary_topic_3']

sentiment_df[topic_cols] = sentiment_df[topic_cols].fillna('Missing')

# get unique list of labels accross relevant columns
labels = pd.unique(sentiment_df[topic_cols].values.ravel('K'))

# build dict with label names as required by AutoML for NLP multilabel
replace_values = dict((label, '') for label in labels)

for key, value in replace_values.items():
    new_value = key.lower().replace(' ', '_').replace('/', '').replace('__','_')
    replace_values[key] = new_value

# replace_values
'''

In [8]:
'''
def generate_multilabel_col(col, no_value='Missing'):
    
    multilabel = [replace_values[label] for label in [col[0], col[1], col[2], col[3], col[4], col[5]] if label != no_value]

    return chr(34) + str(multilabel) + chr(34)

sentiment_df['multilabel'] = sentiment_df[topic_cols].apply(generate_multilabel_col, axis = 1)

# sentiment_df
'''

## Option B: for AML Labeling Export (MS annotated dataset)

In [2]:
sent_rel_df = pd.read_csv('../../input_data/andreas-labelled-raw.csv')

In [3]:
sent_rel_df.sample(5)

,index,comments,Label,LabelConfidence
3255,17257220,Onion rings were soggy and dripping with grease. The burger was good but the onion rings will get you shut down by the health board.,"Bad quality or taste,Good food quality,Good taste","1,1,1"
4382,17249909,Both tacos fell apart because they were stuck to bottom of container.,"Bad quality or taste,Inappropriate packaging","1,1"
1509,17257627,Poor,Global negative feedbacks,1
3980,17248788,A little sad that I only got 2 ribs instead of 5.,"Wrong or missing order,Portion too small","1,1"
2378,17252960,Good. Thanks,Global positive feedbacks,1


In [4]:
# get unique list of labels and drop entries with multiple labels in one line (separated by comma)
labels = list(pd.unique(sent_rel_df['Label']))
labels = list(set([element for element in labels if ',' not in element]))

# combine 'good food quality' and 'good taste' to a single label 'good quality or taste'
gf = 'Good food quality'
gt = 'Good taste'
gqt = 'Good quality or taste'

labels.remove(gf)
labels = [gqt if item == gt else item for item in labels]

# build dict with label names according to the naming conventions of AutoML for NLP
replace_values = dict((label, '') for label in labels)

for key, value in replace_values.items():
    new_value = key.lower().replace(' ', '_')
    replace_values[key] = new_value

replace_values

{'Bad service': 'bad_service',
 'Misleading images': 'misleading_images',
 'Waited too long': 'waited_too_long',
 'Global negative feedbacks': 'global_negative_feedbacks',
 'Good services': 'good_services',
 'Product not available': 'product_not_available',
 'App works well': 'app_works_well',
 'Wrong or missing order': 'wrong_or_missing_order',
 'App to improve': 'app_to_improve',
 'Good food temperature': 'good_food_temperature',
 'Inappropriate packaging': 'inappropriate_packaging',
 'Portion too small': 'portion_too_small',
 'Not enough options': 'not_enough_options',
 'Bad quality or taste': 'bad_quality_or_taste',
 'Global positive feedbacks': 'global_positive_feedbacks',
 'Good quality or taste': 'good_quality_or_taste',
 'Too expensive': 'too_expensive',
 'Other': 'other',
 'Arrived on time': 'arrived_on_time',
 'Bad food temperature': 'bad_food_temperature',
 'Packaging not sustainable': 'packaging_not_sustainable'}

In [5]:
def adjust_labels(input):
    input_list = input.split(',')

    # combine 'good food quality' and 'good taste' to a single label 'good quality or taste'
    if gf in input_list or gt in input_list:
        input_list.append(gqt)
        if gf in input_list:
            input_list.remove(gf)
        if gt in input_list:
            input_list.remove(gt)

    # rename labels according to AutoML for NLP convention
    shortnames = [replace_values[item] for item in input_list]
    
    # add double quotes (chr 34) to the whole list and convert to string
    result = chr(34) + str(shortnames) + chr(34)
    
    return result

sent_rel_df['multilabel'] = sent_rel_df['Label'].apply(adjust_labels)

In [29]:
# remove special characters from comments
def remove_special_chars(text):
    regex = '[^0-9a-zA-Z$£!.,:;/& ]+'
    result = re.sub(regex, '', text)
    return result

# remove_special_chars('a£b c$def!')

sent_rel_df['comments'] = sent_rel_df['comments'].apply(remove_special_chars)


In [7]:
# Generate train and validation splits
cols_of_interest = ['comments', 'multilabel']

# train_df, val_df = train_test_split(sentiment_df[cols_of_interest], test_size=0.2, stratify=sentiment_df['sentiment'], random_state=123)
train_df, val_df = train_test_split(sent_rel_df[cols_of_interest], test_size=0.2, random_state=123)

In [8]:
# review of preprocessed data
train_df.sample(20)

,comments,multilabel
2792,Could have been warmer.,"""['bad_food_temperature']"""
2872,On time and everything was there,"""['arrived_on_time']"""
1961,"I ordered a bacon sandwich w/ a side sausage patty. Only received the patty. When I mentioned this I was told ""that's how it came through"" I showed my receipt and received the same response until the manager got involved and told them to make the sandwich","""['wrong_or_missing_order', 'bad_service']"""
1007,Great stuff!,"""['global_positive_feedbacks']"""
4424,Felt like it didn’t have a lot of flavor. More masa than anything,"""['bad_quality_or_taste']"""
1423,"It took a lot of time to load the app, however it eventually worked.","""['app_to_improve']"""
1933,Apps are too complicated…,"""['app_to_improve']"""
2222,Forgot my bacon.,"""['wrong_or_missing_order']"""
3164,Always great service with Fiona,"""['good_services']"""
3390,My food isn't in the box,"""['wrong_or_missing_order']"""


In [9]:
# train_df.to_csv('../../input_data/ms_sentiment_multilabel_train.csv', index = False)
# val_df.to_csv('../../input_data/ms_sentiment_multilabel_val.csv', index = False)

In [32]:
train_df.to_csv('./sentiment_multilabel_train_mltable/sentiment_multilabel_train.csv', index = False)

val_df.to_csv('./sentiment_multilabel_val_mltable/sentiment_multilabel_val.csv', index = False)

# 2. Configure AutoML for NLP Job

In [2]:
credential = DefaultAzureCredential(exclude_interactive_browser_credential=False)
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = '2dbd7833-129e-4a48-b976-e6dd28a92c29'
    resource_group = 'sodexo-nlp'
    workspace = 'sodexo-nlp'
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)

We could not find config.json in: . or in its parent directories. 


In [3]:
workspace = ml_client.workspaces.get(name=ml_client.workspace_name)

output = {}
output["Workspace"] = ml_client.workspace_name
output["Subscription ID"] = ml_client.connections._subscription_id
output["Resource Group"] = workspace.resource_group
output["Location"] = workspace.location
output

{'Workspace': 'sodexo-nlp',
 'Subscription ID': '2dbd7833-129e-4a48-b976-e6dd28a92c29',
 'Resource Group': 'sodexo-nlp',
 'Location': 'westeurope'}

In [4]:
# general job parameters
compute_name = "gpu-cluster"
exp_name = "automl-nlp-sentiment-multilabel-classification"

training_data_path = './sentiment_multilabel_train_augmented_mltable/' # AUGMENTED !!!
validation_data_path = './sentiment_multilabel_val_mltable/'

# Training MLTable defined locally, with local data to be uploaded
my_training_data_input = Input(type=AssetTypes.MLTABLE, path=training_data_path)

# Validation MLTable defined locally, with local data to be uploaded
my_validation_data_input = Input(type=AssetTypes.MLTABLE, path=validation_data_path)

# Create the AutoML job with the related factory-function.
text_classification_job = automl.text_classification_multilabel(
    compute=compute_name,
    # name="sdk multilabel",
    experiment_name=exp_name,
    training_data=my_training_data_input,
    validation_data=my_validation_data_input,
    target_column_name="multilabel",
    primary_metric="accuracy",
    tags={"task": "sentiment-multilabel-classification-sdkv2"},
)

text_classification_job.set_limits(timeout_minutes=120)
text_classification_job.set_featurization(dataset_language='eng')

Class TextClassificationMultilabelJob: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [5]:
returned_job = ml_client.jobs.create_or_update(
    text_classification_job
)  # submit the job to the backend

print(f"Created job: {returned_job}")
ml_client.jobs.stream(returned_job.name)

Uploading sentiment_multilabel_train_augmented_mltable (0.34 MBs): 100%|██████████| 341085/341085 [00:00<00:00, 15110153.04it/s]


Readonly attribute primary_metric will be ignored in class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.TextClassificationMultilabel'>


Created job: TextClassificationMultilabelJob({'task_type': <TaskType.TEXT_CLASSIFICATION_MULTILABEL: 'TextClassificationMultilabel'>, 'environment_id': None, 'environment_variables': None, 'outputs': {}, 'type': 'automl', 'status': 'NotStarted', 'log_files': None, 'name': 'great_goat_fq9lxr6rjz', 'description': None, 'tags': {'task': 'sentiment-multilabel-classification-sdkv2'}, 'properties': {'mlflow.source.git.repoURL': 'https://github.com/andreaskopp/sodexo-nlp', 'mlflow.source.git.branch': 'main', 'mlflow.source.git.commit': '603d681674fa1c9809bbe16bf1ecb5659af74dcb', 'azureml.git.dirty': 'True'}, 'id': '/subscriptions/2dbd7833-129e-4a48-b976-e6dd28a92c29/resourceGroups/sodexo-nlp/providers/Microsoft.MachineLearningServices/workspaces/sodexo-nlp/jobs/great_goat_fq9lxr6rjz', 'base_path': './', 'creation_context': <azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.SystemData object at 0x7f3eb9bc5310>, 'serialize': <msrest.serialization.Serializer object at 0x7f3eb9bc5a30

# 3. Test Webservice

In [2]:

val_df = pd.read_csv('./sentiment_multilabel_val_mltable/sentiment_multilabel_val.csv')
train_df = pd.read_csv('./sentiment_multilabel_train_mltable/sentiment_multilabel_train.csv')

In [11]:
train_df.sample(10)

,comments,multilabel
523,Food was cold.,"""['bad_food_temperature']"""
1853,First time using this service and it was quick and easy. Great app,"""['app_works_well', 'arrived_on_time']"""
2150,"On top of being cold, One rid was eatable one was burned and the third was 90 fat!!!","""['bad_food_temperature', 'bad_quality_or_taste']"""
33,chicken was burnt,"""['bad_quality_or_taste']"""
3290,Can we cook shrimps with the rice just like the real paella,"""['bad_quality_or_taste']"""
2494,"I was sceptical on how this food would be, but I have to say if was delicious. Still lovely and warm , crispy batter. Well done all","""['good_food_temperature', 'good_quality_or_taste']"""
1473,Please do this every Friday. So good! An portion was the right size,"""['global_positive_feedbacks']"""
2088,Chicken noodles soup did not have any flavor.,"""['bad_quality_or_taste']"""
3580,Bring the sandwich bar back!,"""['not_enough_options']"""
1953,Brill,"""['global_positive_feedbacks']"""


In [4]:
data = val_df[['comments']].to_dict(orient='records')

scoring_uri = 'http://20.238.236.162:80/api/v1/service/sentiment-multilabel-nochar/score'
key = 'YbdlEAoJmatJHdnOyOq778hadKMYDFf1'

In [5]:
# Set the appropriate headers
headers = {"Content-Type": "application/json"}
headers["Authorization"] = f"Bearer {key}"

# Make the request and display the response and logs
data_dict = {"Inputs": {"data": data},
            "GlobalParameters": {"method": "predict"}}

data = json.dumps(data_dict)
resp = requests.post(scoring_uri, data=data, headers=headers)

In [6]:
resp

<Response [200]>

In [7]:
# resp.text

In [13]:
y_true_series = val_df['multilabel']
y_pred_list = json.loads(resp.text)['Results']

# get labels from original dataset
labels = [value for value in replace_values.values()] 

y_pred_arr = np.zeros([len(y_pred_list), len(labels)])
y_true_arr = np.zeros([len(y_true_series), len(labels)])

# build encoded matrix for y_pred_arr
for row_idx, observation in enumerate(y_pred_list):
    for label_idx, label in enumerate(labels):
        if label in observation:
            y_pred_arr[row_idx, label_idx] = 1

# build encoded matrix for y_true_arr
for row_idx, observation in enumerate(y_true_series):
    for label_idx, label in enumerate(labels):
        if observation.find(label) != -1:
            y_true_arr[row_idx, label_idx] = 1


In [14]:
val_df

,comments,multilabel
0,Great staff and the sandwich tasted,"""['good_services', 'good_quality_or_taste']"""
1,Great service from Adrian,"""['good_services']"""
2,Excellent experience. Thanks for the wonderful Meals every time I ordered. I really them.,"""['global_positive_feedbacks', 'good_quality_or_taste']"""
3,Food was just done as I got there and the cook was very friendly. The burger also tasted great!,"""['good_services', 'arrived_on_time', 'good_quality_or_taste']"""
4,Great service from Callum!,"""['good_services']"""
...,...,...
893,"Plz clean ketchup packets, they are sticky","""['bad_service']"""
894,Great first experience. Took a bit long to set up the app but next time it will be easier. :,"""['global_positive_feedbacks']"""
895,"Please include the default toppings so in future orders I can ask them to hold these toppings i.e. lettuce, tomato, onions, etc.","""['app_to_improve']"""
896,Food was amazing. You guys are awesome. Have a great day.,"""['good_services', 'good_quality_or_taste']"""


In [13]:
print(classification_report(y_true_arr, y_pred_arr, target_names=labels, zero_division=True))

                           precision    recall  f1-score   support

            good_services       0.98      0.70      0.81       174
        misleading_images       1.00      0.00      0.00         5
                    other       1.00      0.00      0.00        21
   wrong_or_missing_order       1.00      0.29      0.45       126
    good_food_temperature       1.00      0.00      0.00         8
    product_not_available       1.00      0.00      0.00        12
global_negative_feedbacks       1.00      0.00      0.00        37
    good_quality_or_taste       0.97      0.81      0.88       229
     bad_quality_or_taste       0.84      0.72      0.78       136
       not_enough_options       1.00      0.00      0.00        43
              bad_service       1.00      0.00      0.00        17
  inappropriate_packaging       1.00      0.00      0.00        23
            too_expensive       1.00      0.00      0.00         8
           app_works_well       1.00      0.00      0.00     

In [14]:
val_df['predicted'] = y_pred_list

In [15]:
val_df.sample(50)

,comments,multilabel,predicted
839,Cafeteria food is definitely improved,"""['good_quality_or_taste']""",[bad_quality_or_taste]
744,Came in plastic and wasnt sure if I could microwave it. Did I need to bring my own plate App said order would take 2hrs. Wanted to cancel but had paid and wasnt sure if it could be,"""['app_to_improve', 'waited_too_long']""",[]
729,Eggs were fluffy and bacon crispy. Great job! Thank you,"""['good_quality_or_taste']""",[good_quality_or_taste]
842,No sour cream or salsa came with burrito per request.,"""['wrong_or_missing_order']""",[]
141,First app order when collecting order was told no app orders are received on nights fortunately someone else had changed their mind and a spare order was available.,"""['wrong_or_missing_order', 'app_to_improve']""",[]
690,"Missing soup. Salad is bland, would be nice to have good mix of vegetables instead. Main meal meat is good, but rice is bland. Overall, portion sizes are too small.","""['bad_quality_or_taste', 'not_enough_options', 'portion_too_small']""",[bad_quality_or_taste]
193,I enjoy the food but I am always disappointed that the pictures you use for each item do not match what it really looks like. It is misleading the pictures contain more ingredients than you use and I feel unhappy that I did not get what the picture was.,"""['misleading_images', 'good_quality_or_taste']""",[]
831,"No sausages on bap, no knife to spread jam","""['wrong_or_missing_order', 'global_negative_feedbacks']""",[]
325,Fries were not cooked enough. Third time in a row. Felt like I was eating raw potatoes. The fries used to be so good.,"""['bad_quality_or_taste']""",[bad_quality_or_taste]
347,great service! easy!!,"""['good_services']""",[good_services]


In [16]:
val_df['predicted'].value_counts()

[]                                               377
[good_quality_or_taste]                          166
[bad_quality_or_taste]                           113
[good_services]                                  101
[global_positive_feedbacks]                       78
[wrong_or_missing_order]                          37
[good_quality_or_taste, good_services]            23
[bad_quality_or_taste, good_quality_or_taste]      3
Name: predicted, dtype: int64